In [36]:
from Bio import SeqIO, Entrez
import os


In [37]:
def affichage(sequence,seg_list):
    print(f"id :{sequence.id}")
    print(f"longeur : {len(sequence.seq)}")
    seg_list.append(sequence.seq)
    print(f"séquence : {sequence.seq}")

In [38]:
def explore_alignments(i, j, current_align1, current_align2,alignments,score_matrix,seq1,seq2):
    # Condition d'arrêt
    if i < 2 and j < 2 :
        alignments.append((current_align1, current_align2))
        return

    diag = score_matrix[i - 1][j - 1] if i > 0 and j > 0 else float('-inf')
    up = score_matrix[i - 1][j] if i > 0 else float('-inf')
    left = score_matrix[i][j - 1] if j > 0 else float('-inf')

    max_value = max(diag, up, left)
    if max_value == diag:
        if i>1 and j>1 :
        
            explore_alignments(i - 1, j - 1, seq1[i -2] + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)
        elif j<1 : 
            explore_alignments(i - 1, j - 1, seq1[i -2] + current_align1, '-' + current_align2,alignments,score_matrix,seq1,seq2)
        elif i<1:
             explore_alignments(i - 1, j - 1, '-' + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)

    if max_value == up:
        explore_alignments(i - 1, j, seq1[i -2] + current_align1, '-' + current_align2,alignments,score_matrix,seq1,seq2)
    if max_value == left:
        explore_alignments(i, j - 1, '-' + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)

    return alignments    


In [39]:
def global_alignment(seq1, seq2, match=2, mismatch=-1, gap=-1):

    n = len(seq1) + 1
    m = len(seq2) + 1

    score_matrix = [[0] * m for _ in range(n)]
    print(score_matrix)
    # traceback_matrix = [[None] * m for _ in range(n)]

    # Rempli gap
    for i in range(1, n):
        score_matrix[i][0] = score_matrix[i - 1][0] + gap
        # traceback_matrix[i][0] = 'up'
    for j in range(1, m):
        score_matrix[0][j] = score_matrix[0][j - 1] + gap
        # traceback_matrix[0][j] = 'left'


    for i in range(1, n):
        for j in range(1, m):
            if seq1[i - 1] == seq2[j - 1]:
                score = match
            else:
                score = mismatch

            #best score
            diag = score_matrix[i - 1][j - 1] + score
            up = score_matrix[i - 1][j] + gap
            left = score_matrix[i][j - 1] + gap
            max_score = max(diag, up, left)
            score_matrix[i][j] = max_score

            # if max_score == diag:
            #     traceback_matrix[i][j] = 'diag'
            # elif max_score == up:
            #     traceback_matrix[i][j] = 'up'
            # else:
            #     traceback_matrix[i][j] = 'left'

    alignments = []  
    i, j = n - 1, m - 1
    initial_align1 = seq1[i-1]  
    initial_align2 = seq2[j-1]  

    alignments = explore_alignments(i, j, initial_align1, initial_align2,alignments,score_matrix,seq1,seq2)
                    
    print("Matrice de score :")
    for row in score_matrix:
        print(row)    
    
    return alignments, score_matrix[n - 1][m - 1]




In [40]:
def local_alignment(seq1, seq2, match, mismatch, gap):
    n = len(seq1) + 1
    m = len(seq2) + 1

    score_matrix = [[0] * m for _ in range(n)]

    # max_pos = (0, 0)
    
    for i in range(1, n):
        for j in range(1, m):
            if seq1[i - 1] == seq2[j - 1]:
                score = match
            else:
                score = mismatch

            diag = score_matrix[i - 1][j - 1] + score
            up = score_matrix[i - 1][j] + gap
            left = score_matrix[i][j - 1] + gap
            
            score_matrix[i][j] = max(0,diag, up, left)
            
            # # Track maximum score position
            # if score_matrix[i][j] > max_score:
            #     max_score = score_matrix[i][j]

    End = score_matrix[n - 1][m - 1]
    diag = score_matrix[n - 2][m - 2]
    up = score_matrix[n - 2][m-1]
    left = score_matrix[n-1][m - 2]
    score_final = max(diag,up,left,End) 

    if  score_final == End:
        i, j = n - 1, m - 1
        initial_align1 = seq1[i-1]  
        initial_align2 = seq2[j-1]
    elif   score_final == diag:
        i, j = n - 2, m - 1
        initial_align1 = seq1[i-2]  
        initial_align2 = seq2[j-2]        
    elif   score_final == up:
        i, j = n - 2, m - 1
        initial_align1 = seq1[i-2]  
        initial_align2 = seq2[j-1]
    elif score_final == left: 
        i, j = n - 1, m - 2 
        initial_align1 = seq1[i-1]  
        initial_align2 = seq2[j-2]   


    alignments = []  
      

    alignments = explore_alignments(i, j, initial_align1, initial_align2,alignments,score_matrix,seq1,seq2)
       
    print("Matrice de score :")
    for row in score_matrix:
        print(row)
    
    return alignments, score_final


In [41]:
def calculate_score(align1, align2, match, mismatch, gap):
    score = 0
    for a, b in zip(align1, align2):
        if a == '-' or b == '-':
            score += gap
        elif a == b:
            score += match
        else:
            score += mismatch
    return score

In [42]:
# Exemple d'utilisation
seq1 = "CTTG"
seq2 = "ACTGG"
alignments, score = global_alignment(seq1, seq2,2,-1,-1)
print("Score de la matrice :", score)
print("Alignement de la séquence 1 :", alignments)
scored_alignments = [(align1, align2, f'score = {calculate_score(align1, align2,2,-1,-1)}' ) for align1, align2 in alignments]
print(scored_alignments)


[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
Matrice de score :
[0, -1, -2, -3, -4, -5]
[-1, -1, 1, 0, -1, -2]
[-2, -2, 0, 3, 2, 1]
[-3, -3, -1, 2, 2, 1]
[-4, -4, -2, 1, 4, 4]
Score de la matrice : 4
Alignement de la séquence 1 : [('-CTT-G', 'AC-TGG'), ('-CTT-G', 'ACT-GG')]
[('-CTT-G', 'AC-TGG', 'score = 3'), ('-CTT-G', 'ACT-GG', 'score = 3')]


In [43]:
alignments, score = local_alignment(seq1, seq2,2,-1,-2)
print("Score de la matrice :", score)
print("Alignement de la séquence 1 :", alignments)
scored_alignments = [(align1, align2, f'score = {calculate_score(align1, align2,2,-1,-2)}' ) for align1, align2 in alignments]
print(scored_alignments)


Matrice de score :
[0, 0, 0, 0, 0, 0]
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 4, 2, 0]
[0, 0, 0, 2, 3, 1]
[0, 0, 0, 0, 4, 5]
Score de la matrice : 5
Alignement de la séquence 1 : [('-GCTT-G', 'A-CT-GG'), ('-CTT-G', 'ACT-GG')]
[('-GCTT-G', 'A-CT-GG', 'score = -2'), ('-CTT-G', 'ACT-GG', 'score = 0')]


In [1]:
sequences = [([0], [1]), ([2], [0, 1])] 
labels = [f"I{i+1}" for i in range(len(sequences))]
print(labels)

['I1', 'I2']


In [3]:
sequences = ["ACTGG", "ACTTGG", "ACTGC", "CTTG"]  
labels = [i for i in range(len(sequences))]
print(labels)

[0, 1, 2, 3]
